# Applying Advanced Transformations (Core)
## The Data
You will be working with a heavily modified version of the Superheroes dataset from Kaggle.

The dataset includes two csv's:

superhero_info.csv:
Contains Name, Publisher, Demographic Info, and Body measurements.

superhero_powers.csv:
Contains Hero name and list of powers

### The Task
Your task is two-fold:

**I. Clean the files and combine them into one final DataFrame.**

 This dataframe should have the following columns:

* Hero (Just the name of the Hero)

* Publisher

* Gender

* Eye color

* Race

* Hair color

* Height (numeric)

* Skin color

* Alignment

* Weight (numeric)

    Plus, one-hot-encoded columns for every power that appears in the dataset. 
   
    E.g.:
    * Agility
    * Flight
    * Superspeed
    * etc.

Hint: There is a space in "100 kg" or "52.5 cm"



**II. Use your combined DataFrame to answer the following questions.**

    Compare the average weight of super powers who have Super Speed to those who do not.

    What is the average height of heroes for each publisher?

In [1]:
# Loading in libraries
import pandas as pd
import json

In [2]:
# Loading data
hero_info=pd.read_csv('superhero_info - superhero_info.csv')
hero_powers=pd.read_csv('superhero_powers - superhero_powers.csv')
# previewing
display(hero_info.head())
display(hero_powers.head())

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [3]:
hero_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [4]:
hero_powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


# Hero_info

In [5]:
# Creating separate 'Hero' and 'Publisher' columns from 'Hero|Publisher' column
hero_info[['Hero','Publisher']]=hero_info['Hero|Publisher'].str.split('|',expand=True)
# Dropping old column
hero_info=hero_info.drop(columns='Hero|Publisher')
hero_info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics


In [6]:
# Checking a row of 'Measurements' column before attempting to apply json.loads
hero_info['Measurements'][0]

"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [7]:
# double quotes need to be replaced with single quotes before applying json.loads 
hero_info['Measurements']=hero_info['Measurements'].str.replace("'",'"')
hero_info['Measurements'][0]

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [8]:
# applying json.loads to convert all strings into actual dictionaries
hero_info['Measurements']=hero_info['Measurements'].apply(json.loads)
hero_info['Measurements'][0]

{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [9]:
# applying .Series to 'Measurements' to get 'Height' and 'Weight' columns
height_weight=hero_info['Measurements'].apply(pd.Series)

In [10]:
# adding columns to hero_info dataframe
hero_info=pd.concat([hero_info,height_weight], axis=1)
hero_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [11]:
# dropping redundant column
hero_info=hero_info.drop(columns='Measurements')
hero_info.columns

Index(['Gender', 'Race', 'Alignment', 'Hair color', 'Eye color', 'Skin color',
       'Hero', 'Publisher', 'Height', 'Weight'],
      dtype='object')

In [12]:
# Now we can remove 'cm' and 'kg' from 'Height' and 'Weight' before turning them in to numeric columns
hero_info['Height']=hero_info['Height'].str.replace('cm','')
hero_info['Weight']=hero_info['Weight'].str.replace('kg','')
hero_info[['Height','Weight']].head()

,Height,Weight
0,203.0,441.0
1,191.0,65.0
2,185.0,90.0
3,203.0,441.0
4,193.0,122.0


In [13]:
# correcting datatypes
hero_info['Weight']=hero_info['Weight'].astype('float64')
hero_info['Height']=hero_info['Height'].astype('float64')
hero_info.info() # checking 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      463 non-null    object 
 1   Race        463 non-null    object 
 2   Alignment   463 non-null    object 
 3   Hair color  463 non-null    object 
 4   Eye color   463 non-null    object 
 5   Skin color  463 non-null    object 
 6   Hero        463 non-null    object 
 7   Publisher   463 non-null    object 
 8   Height      463 non-null    float64
 9   Weight      463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


# Hero_powers

In [14]:
hero_powers.head # checking 

<bound method NDFrame.head of           hero_names                                             Powers
0            3-D Man         Agility,Super Strength,Stamina,Super Speed
1             A-Bomb  Accelerated Healing,Durability,Longevity,Super...
2         Abe Sapien  Agility,Accelerated Healing,Cold Resistance,Du...
3           Abin Sur                                 Lantern Power Ring
4        Abomination  Accelerated Healing,Intelligence,Super Strengt...
..               ...                                                ...
662  Yellowjacket II                 Flight,Energy Blasts,Size Changing
663             Ymir  Cold Resistance,Durability,Longevity,Super Str...
664             Yoda  Agility,Stealth,Danger Sense,Marksmanship,Weap...
665          Zatanna  Cryokinesis,Telepathy,Magic,Fire Control,Proba...
666             Zoom  Super Speed,Intangibility,Time Travel,Time Man...

[667 rows x 2 columns]>

In [15]:
# checking object type for 'Powers' column
hero_powers['Powers'][0]

'Agility,Super Strength,Stamina,Super Speed'

In [16]:
test=hero_powers['Powers'][0]
test

'Agility,Super Strength,Stamina,Super Speed'

In [17]:
test=test.replace(',','","')
test

'Agility","Super Strength","Stamina","Super Speed'

In [18]:
test='["' + test + '"]'
test

'["Agility","Super Strength","Stamina","Super Speed"]'

In [19]:
# creating lists from strings in 'Powers' column by adding double quotes before and after all commas
hero_powers['Powers']=hero_powers['Powers'].str.replace(',','","')
hero_powers['Powers_split'] = '["' + hero_powers['Powers'] + '"]'
hero_powers['Powers_split'].head()

0    ["Agility","Super Strength","Stamina","Super S...
1    ["Accelerated Healing","Durability","Longevity...
2    ["Agility","Accelerated Healing","Cold Resista...
3                               ["Lantern Power Ring"]
4    ["Accelerated Healing","Intelligence","Super S...
Name: Powers_split, dtype: object

In [20]:
# dropping double quotes while keeping list-form
hero_powers['Powers_split']=hero_powers['Powers_split'].apply(json.loads)
hero_powers['Powers_split'].head()


0      [Agility, Super Strength, Stamina, Super Speed]
1    [Accelerated Healing, Durability, Longevity, S...
2    [Agility, Accelerated Healing, Cold Resistance...
3                                 [Lantern Power Ring]
4    [Accelerated Healing, Intelligence, Super Stre...
Name: Powers_split, dtype: object

In [21]:
# 'one hot encoding' 'Powers_split' column using .explode()
exploded=hero_powers.explode('Powers_split')
exploded.head(3)

,hero_names,Powers,Powers_split
0,3-D Man,"Agility"",""Super Strength"",""Stamina"",""Super Speed",Agility
0,3-D Man,"Agility"",""Super Strength"",""Stamina"",""Super Speed",Super Strength
0,3-D Man,"Agility"",""Super Strength"",""Stamina"",""Super Speed",Stamina


In [22]:
# dropping nan values before grabbing unique values
cols=exploded['Powers_split'].dropna().unique()
cols

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [23]:
# adding encoded columns to 'hero_powers' using a for loop and .str.contains
for col in cols:
    hero_powers[col]=hero_powers['Powers'].str.contains(col)#indicating a true or false for powers

/var/folders/w6/t6jhm8jd0pnbj_5hvcl1nlsw0000gn/T/ipykernel_68997/1500413110.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hero_powers[col]=hero_powers['Powers'].str.contains(col)#indicating a true or false for powers
/var/folders/w6/t6jhm8jd0pnbj_5hvcl1nlsw0000gn/T/ipykernel_68997/1500413110.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hero_powers[col]=hero_powers['Powers'].str.contains(col)#indicating a true or false for powers
/var/folders/w6/t6jhm8jd0pnbj_5hvcl1nlsw0000gn/T/ipykernel_68997/1500413110.py:3: Perfor

,hero_names,Powers,Powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility"",""Super Strength"",""Stamina"",""Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing"",""Durability"",""Longevity"",...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility"",""Accelerated Healing"",""Cold Resistanc...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing"",""Intelligence"",""Super Str...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [25]:
# dropping unnecessary 'Powers' column
hero_powers=hero_powers.drop(columns=['Powers','Powers_split'])
hero_powers.head()

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
hero_powers.head(2)

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


# Merging DataFrames

In [31]:
superheroes=pd.merge(hero_info,hero_powers,left_on='Hero',right_on='hero_names')
# dropping unnecessary 'hero_names' column
superheroes=superheroes.drop(columns='hero_names')
superheroes.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


## Compare the average weight of super powers who have Super Speed to those who do not.


In [36]:
super_speed=superheroes.groupby('Super Speed')['Weight'].mean().round()
super_speed

Super Speed
False    102.0
True     129.0
Name: Weight, dtype: float64

## What is the average height of heroes for each publisher?

In [40]:
avg_height=superheroes.groupby('Publisher')['Height'].mean().round(2)
avg_height

Publisher
DC Comics            181.92
Dark Horse Comics    176.91
George Lucas         159.60
Image Comics         211.00
Marvel Comics        191.55
Shueisha             171.50
Star Trek            181.50
Team Epic TV         180.75
Unknown              178.00
Name: Height, dtype: float64